In [1]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

In [ ]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
#model = trt_pose.models.densenet121_baseline_att(num_parts, 2 * num_links).cuda().eval()

In [ ]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'
#MODEL_WEIGHTS = 'densenet121_baseline_att_256x256_B_epoch_160.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

In [ ]:
WIDTH = 224
HEIGHT = 224

#WIDTH = 256
#HEIGHT = 256

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

In [ ]:
import time
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)
resize = transforms.Compose([transforms.ToPILImage(),transforms.Resize((HEIGHT, WIDTH))])

In [ ]:
vid = cv2.VideoCapture('HumanPose1.mp4')
vidRunning = True
    
while(vidRunning):
    tup, frame = vid.read()
    if tup:
        resize(frame)
        data = preprocess(frame)
        t0 = time.time()
        cmap, paf = model(data)
        t1 = time.time()
        print("This took ", (t1-t0))
        cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
        counts, objects, peaks = parse_objects(cmap, paf)
        draw_objects(frame, counts, objects, peaks)
        cv2.imshow('TRT_OUTPUT', frame)
        cv2.waitKey(1)
    else:
        vidRunning = False